***
# SPARK VERSION FOR DEEQU
***

# Feature transformation with Amazon SageMaker Processing and SparkML


Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Spark are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Spark in a managed SageMaker environment to run our preprocessing workload. Then, we'll take our preprocessed dataset and train a regression model using XGBoost.

![](images/processing.jpg)


In [5]:
import sagemaker
from time import gmtime, strftime

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()

timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# TODO:  Clean these up
#input_raw_prefix = 'sagemaker/spark-preprocess-reviews-demo/input/raw/reviews'
output_prefix = 'amazon-reviews-spark-analyzer-{}'.format(timestamp_prefix)
#model_prefix = 'sagemaker/spark-preprocess-reviews-demo/model'
processing_job_name = 'amazon-reviews-spark-analyzer-{}'.format(timestamp_prefix)

print('Processing job name:  {}'.format(processing_job_name))

Processing job name:  amazon-reviews-spark-analyzer-2020-02-25-21-49-56


## Using Amazon SageMaker Processing to execute a SparkML job

### Downloading dataset and uploading to Amazon Simple Storage Service (Amazon S3)

Show the dataset

In [6]:
#!aws s3 ls s3://$bucket/$input_raw_prefix/
!aws s3 ls s3://amazon-reviews-pds/parquet/

                           PRE product_category=Apparel/
                           PRE product_category=Automotive/
                           PRE product_category=Baby/
                           PRE product_category=Beauty/
                           PRE product_category=Books/
                           PRE product_category=Camera/
                           PRE product_category=Digital_Ebook_Purchase/
                           PRE product_category=Digital_Music_Purchase/
                           PRE product_category=Digital_Software/
                           PRE product_category=Digital_Video_Download/
                           PRE product_category=Digital_Video_Games/
                           PRE product_category=Electronics/
                           PRE product_category=Furniture/
                           PRE product_category=Gift_Card/
                           PRE product_category=Grocery/
                           PRE product_category=Health_&_Personal_Care/
   

### Build a Spark container for running the preprocessing job

An example Spark container is included in the `./container` directory of this example. The container handles the bootstrapping of all Spark configuration, and serves as a wrapper around the `spark-submit` CLI. At a high level the container provides:
* A set of default Spark/YARN/Hadoop configurations
* A bootstrapping script for configuring and starting up Spark master/worker nodes
* A wrapper around the `spark-submit` CLI to submit a Spark application


After the container build and push process is complete, use the Amazon SageMaker Python SDK to submit a managed, distributed Spark application that performs our dataset preprocessing.

Build the example Spark container.

In [40]:
!cat container/Dockerfile

FROM openjdk:8-jre-slim

RUN apt-get update
RUN apt-get install -y curl unzip python3 python3-setuptools python3-pip python-dev python3-dev python-psutil
RUN pip3 install py4j psutil==5.6.5 numpy==1.17.4
RUN apt-get clean
RUN rm -rf /var/lib/apt/lists/*

# http://blog.stuart.axelbrooke.com/python-3-on-spark-return-of-the-pythonhashseed
ENV PYTHONHASHSEED 0
ENV PYTHONIOENCODING UTF-8
ENV PIP_DISABLE_PIP_VERSION_CHECK 1

# Install Hadoop
ENV HADOOP_VERSION 3.0.0
ENV HADOOP_HOME /usr/hadoop-$HADOOP_VERSION
ENV HADOOP_CONF_DIR=$HADOOP_HOME/etc/hadoop
ENV PATH $PATH:$HADOOP_HOME/bin
RUN curl -sL --retry 3 \
  "http://archive.apache.org/dist/hadoop/common/hadoop-$HADOOP_VERSION/hadoop-$HADOOP_VERSION.tar.gz" \
  | gunzip \
  | tar -x -C /usr/ \
 && rm -rf $HADOOP_HOME/share/doc \
 && chown -R root:root $HADOOP_HOME

# Install Spark
ENV SPARK_VERSION 2.4.5
ENV SPARK_PACKAGE spark-${SPARK_VERSION}-bin-without-hadoop
ENV SPARK_HOME /usr/spark-${SPARK_VERSION}
ENV SPARK_DIST_CLASSPATH="$HADOOP_H

In [70]:
docker_repo = 'amazon-reviews-spark-analyzer'
docker_tag = 'latest'

In [71]:
!docker build -t $docker_repo:$docker_tag -f container/Dockerfile ./container

Sending build context to Docker daemon  3.022MB
Step 1/33 : FROM openjdk:8-jre-slim
 ---> 9c82c74fbc96
Step 2/33 : RUN apt-get update
 ---> Using cache
 ---> ca0a6099c443
Step 3/33 : RUN apt-get install -y curl unzip python3 python3-setuptools python3-pip python-dev python3-dev python-psutil
 ---> Using cache
 ---> cc1fd71bd88c
Step 4/33 : RUN pip3 install py4j psutil==5.6.5 numpy==1.17.4
 ---> Using cache
 ---> 376699e73ced
Step 5/33 : RUN apt-get clean
 ---> Using cache
 ---> 1ea11fb14632
Step 6/33 : RUN rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> ea880623fd34
Step 7/33 : ENV PYTHONHASHSEED 0
 ---> Using cache
 ---> 7d1b53453a5e
Step 8/33 : ENV PYTHONIOENCODING UTF-8
 ---> Using cache
 ---> 12cfee88f392
Step 9/33 : ENV PIP_DISABLE_PIP_VERSION_CHECK 1
 ---> Using cache
 ---> 8010c768a0e5
Step 10/33 : ENV HADOOP_VERSION 3.0.0
 ---> Using cache
 ---> 0e10f67d8992
Step 11/33 : ENV HADOOP_HOME /usr/hadoop-$HADOOP_VERSION
 ---> Using cache
 ---> 19f1a57f792c
Step 12/33 : ENV HADOOP

Create an Amazon Elastic Container Registry (Amazon ECR) repository for the Spark container and push the image.

In [72]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, docker_repo, docker_tag)
print(image_uri)

835319576252.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-analyzer:latest


Create ECR repository and push docker image

In [73]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [74]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo

{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:835319576252:repository/amazon-reviews-spark-analyzer",
            "registryId": "835319576252",
            "repositoryName": "amazon-reviews-spark-analyzer",
            "repositoryUri": "835319576252.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-analyzer",
            "createdAt": 1581738843.0,
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            }
        }
    ]
}


In [75]:
!docker tag $docker_repo:$docker_tag $image_uri

In [76]:
!docker push $image_uri

The push refers to repository [835319576252.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-analyzer]

4377c451: Preparing 
268138d8: Preparing 
35559781: Preparing 
6b36b7a4: Preparing 
782557b4: Preparing 
8eb2663a: Preparing 
5ae17e8e: Preparing 
89b8c28a: Preparing 
d604f04b: Preparing 
a936c4d8: Preparing 
f855c32d: Preparing 
964f7673: Preparing 
0d7e7b4a: Preparing 
a6e6c92c: Preparing 
377c451: Pushed       3MBists 5MB5Alatest: digest: sha256:f6fe6ddad0942f0ba99e9298a5e5f39235b3de24ee169a250ac4d51a880f3669 size: 3472


### Run the preprocessing job using Amazon SageMaker Processing

Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built, and a SparkML script for preprocessing in the job configuration.

Review the Spark preprocessing script.

In [77]:
cat preprocess-deequ.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

def main():
    args_iter = iter(sys.argv[1:])
    args = dict(zip(args_iter, args_iter))
    
    # Retrieve the args and replace 's3://' with 's3a://' (used by Spark)
    s3_input_data = args['s3_input_data'].replace('s3://', 's3a://')
    print(s3_input_data)
    s3_output_analyze_data = args['s3_output_analyze_data'].replace('s3://', 's3a://')
    print(s3_output_analyze_data)
    
    spark = SparkSession.builder \
        .appName("Amazon_Reviews_Spark_Analyzer") \
        .getOrCreate()

    # Invoke Main from preprocess-deequ.jar
    getattr(spark._jvm.SparkAmazonReviewsAnalyzer, "run")(s3_input_data, s3_output_analyze_data)

if __name__ == "__main__":
    main()


Run this script as a processing job.  You specify the command (`/opt/program/submit` for this Spark processor.)  You also need to specify one `ProcessingInput` with the `source` argument of the Amazon S3 bucket and `destination` is where the script reads this data from `/opt/ml/processing/input` (inside the Docker container.)  All local paths inside the processing container must begin with `/opt/ml/processing/`.

Also give the `run()` method a `ProcessingOutput`, where the `source` is the path the script writes output data to.  For outputs, the `destination` defaults to an S3 bucket that the Amazon SageMaker Python SDK creates for you, following the format `s3://sagemaker-<region>-<account_id>/<processing_job_name>/output/<output_name>/`.  You also give the `ProcessingOutput` value for `output_name`, to make it easier to retrieve these output artifacts after the job is run.

The arguments parameter in the `run()` method are command-line arguments in our `preprocess.py` script.

In [78]:
from sagemaker.processing import ScriptProcessor

processor = ScriptProcessor(base_job_name='spark-amazon-reviews-analyzer',
                            image_uri=image_uri,
                            command=['/opt/program/submit'],
                            role=role,
                            instance_count=10, # instance_count needs to be > 1 or you will see the following error:  "INFO yarn.Client: Application report for application_ (state: ACCEPTED)"
                            instance_type='ml.r5.8xlarge',
                            max_runtime_in_seconds=600,
                            env={
                                'mode': 'jar',
                                'main_class': 'Main'
                            })

In [79]:
# Inputs
s3_input_data = 's3://amazon-reviews-pds/parquet/'
print(s3_input_data)

s3://amazon-reviews-pds/parquet/


In [80]:
!aws s3 ls $s3_input_data

                           PRE product_category=Apparel/
                           PRE product_category=Automotive/
                           PRE product_category=Baby/
                           PRE product_category=Beauty/
                           PRE product_category=Books/
                           PRE product_category=Camera/
                           PRE product_category=Digital_Ebook_Purchase/
                           PRE product_category=Digital_Music_Purchase/
                           PRE product_category=Digital_Software/
                           PRE product_category=Digital_Video_Download/
                           PRE product_category=Digital_Video_Games/
                           PRE product_category=Electronics/
                           PRE product_category=Furniture/
                           PRE product_category=Gift_Card/
                           PRE product_category=Grocery/
                           PRE product_category=Health_&_Personal_Care/
   

In [81]:
# Outputs
s3_output_analyze_data = 's3://{}/{}/analyze_data'.format(bucket, output_prefix)

print(s3_output_analyze_data)

s3://sagemaker-us-east-1-835319576252/amazon-reviews-spark-analyzer-2020-02-25-21-49-56/analyze_data


In [82]:
# Note:  We can specify the local `preprocess.py` because we're using the SageMaker SDK.
#
#    Notes on Invoking from Lambda:
#      * However, if we use the boto3 SDK (ie. with a Lambda), we need to copy the `preprocess.py` file to S3 and specify the everything include --py-files, etc.
#      * We would need to do the following before invoking the Lambda:
#          !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/code/preprocess.py
#          !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/py_files/preprocess.py
#      * Then reference the s3://<location> above in the --py-files, etc.
#      * See Lambda example code in this same project for more details.
#
# Note:  See https://github.com/awslabs/amazon-sagemaker-examples/issues/994 for issues related to using /opt/ml/processing/input/ and output/
processor.run(code='preprocess-deequ.py',
                    arguments=['s3_input_data', s3_input_data,
                               's3_output_analyze_data', s3_output_analyze_data,
                              ],
                    logs=True,
                    wait=False,
)


Job Name:  spark-amazon-reviews-analyzer-2020-02-26-01-05-05-890
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-amazon-reviews-analyzer-2020-02-26-01-05-05-890/input/code/preprocess-deequ.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  []


In [83]:
#############
# TODO:  CHANGE THE processing_job_name BELOW
#############

region = sagemaker_session.boto_region_name
processing_job_name = 'spark-amazon-reviews-analyzer-2020-02-26-01-05-05-890'


#############
# TODO:  CHANGE THE processing_job_name ABOVE ^^
#############

from IPython.core.display import display, HTML
display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, processing_job_name)))

In [37]:
#!aws sagemaker list-processing-jobs

In [38]:
#running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=processing_job_name,
#                                                                            sagemaker_session=sagemaker_session)
#running_processor.describe()

#### Inspect the processed dataset
Take a look at a few rows of the transformed dataset to make sure the preprocessing was successful.

In [39]:
from IPython.core.display import display, HTML
display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(bucket, output_prefix, region)))

In [78]:
!aws s3 ls --recursive $s3_output_analyze_data/